In [115]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from net import Net
import random
import sys

In [116]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [117]:
x=datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))]))

In [118]:
x.train_labels

tensor([5, 0, 4,  ..., 5, 6, 8])

In [119]:
ran=[i for i in range(0,60000)]
random.seed(42)
index=random.sample(ran,60000//5)

In [120]:
x.train_labels[index].random_(0,10)

tensor([5, 1, 1,  ..., 1, 7, 4])

In [121]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
    parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 256)')
    parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                        help='input batch size for testing (default: 1000)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N',
                        help='number of epochs to train (default: 10)')
    parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                        help='learning rate (default: 0.01)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--no-cuda', action='store_true', default=False,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    
    parser.add_argument('--save-model', action='store_true', default=False,
                        help='For Saving the current Model')
    args = parser.parse_args()
    use_cuda = not args.no_cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")
     
    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        x,
        batch_size=args.batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args.test_batch_size, shuffle=True, **kwargs)
#     train_loader=x

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    for epoch in range(1, args.epochs + 1):
        train(args, model, device, train_loader, optimizer, epoch)
        test(args, model, device, test_loader)

    if (args.save_model):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [122]:
sys.argv = ['-f']
main()

RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/THC/THCGeneral.cpp:74

{tensor(0), tensor(8), tensor(8), tensor(9), tensor(0), tensor(6), tensor(8), tensor(8), tensor(6), tensor(0), tensor(6), tensor(6), tensor(2), tensor(8), tensor(9), tensor(5), tensor(5), tensor(0), tensor(3), tensor(8), tensor(1), tensor(9), tensor(7), tensor(9), tensor(5), tensor(5), tensor(3), tensor(1), tensor(0), tensor(2), tensor(7), tensor(2), tensor(4), tensor(1), tensor(6), tensor(6), tensor(0), tensor(0), tensor(5), tensor(8), tensor(6), tensor(7), tensor(7), tensor(8), tensor(6), tensor(8), tensor(2), tensor(3), tensor(3), tensor(7), tensor(5), tensor(1), tensor(1), tensor(4), tensor(4), tensor(1), tensor(1), tensor(7), tensor(0), tensor(8), tensor(2), tensor(9), tensor(0), tensor(4), tensor(7), tensor(0), tensor(4), tensor(3), tensor(0), tensor(9), tensor(3), tensor(3), tensor(4), tensor(7), tensor(3), tensor(1), tensor(9), tensor(6), tensor(2), tensor(0), tensor(9), tensor(8), tensor(6), tensor(2), tensor(4), tensor(7), tensor(4), tensor(2), tensor(3), tensor(2), tensor(0)